In [147]:
!pip install Faker

In [148]:
import pandas as pd
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
from faker import Faker

In [149]:
# Initialize Faker
fake = Faker()

In [150]:
# Load the model and tokenizer
model_name = "openai-community/gpt2"  # You can choose other models from Hugging Face
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [151]:
#might need
def open_file(filepath):
    with open(filepath, 'r', encoding = 'utf-8') as infile:
        return infile.read()

def save_file(filepath, content):
    with open(filepath, 'w', encoding = 'utf-8') as outfile:
        outfile.write(content)

In [178]:
def generate_synthetic_note(prompt, max_new_tokens=200, temperature=0.7):
    inputs = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=512)
    
    # Generate new tokens
    outputs = model.generate(
        inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True
    )
    
    # Decode only the newly generated tokens
    generated_text = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)
    return generated_text.strip()

In [179]:
def create_json_structure(note):
    return {
        "messages": [
            {
                "role": "system",
                "content": "You are useful and helpful chatbot that helps generate synthetic clinical notes when asked."
            },
            {
                "role": "user",
                "content": note
            }
        ]
    }

In [180]:
def generate_dataset(num_examples=75):
    dataset = []
    prompt = open_file('amanispromp.txt')
    
    for _ in range(num_examples):
        note = generate_synthetic_note(prompt)
        json_structure = create_json_structure(note)
        dataset.append(json_structure)
    return dataset

In [181]:
def save_dataset(dataset, filename='sdoh_dataset.jsonl'):
    with open(filename, 'w') as f:
        for item in dataset:
            json.dump(item, f)
            f.write('\n\n')

In [182]:
# Generate and save the dataset
dataset = generate_dataset(5)
save_dataset(dataset)

print(f"Generated and saved synthetic SDOH notes to sdoh_dataset.jsonl")

# Print a sample of the generated data
print("\nSample of generated data:")
print(json.dumps(dataset[0], indent=2) + '\n\n')

Generated and saved synthetic SDOH notes to sdoh_dataset.jsonl

Sample of generated data:
{
  "messages": [
    {
      "role": "system",
      "content": "You are useful and helpful chatbot that helps generate synthetic clinical notes when asked."
    },
    {
      "role": "user",
      "content": "is a nurse.\n\n\nPt is retired.\n\n\nPersonal history records alcohol use, tobacco use, and\n\ndrug abuse. He is a former medical student in a private\n\nstudio.\n\n\nFamily history records alcohol use, tobacco use, and\n\ndrug abuse. He is a former medical student in a privatestudio.\n\n\nDr. Kavanagh, [**Hospital1 **] at [**Hospital1 **].  Does not have a doctorate.\n\n[**Hospital1 **] does not have a doctorate, and has never\n\nbeen diagnosed with any form of alcohol abuse.\n\n\nFamily history records alcohol use, tobacco use, and\n\ndrug abuse. He is a former medical student in a privatestudio.\n\n\nFamily history records alcohol use, tobacco use, and\n\ndrug abuse. He is a former medi

In [183]:
notee = pd.read_csv("brokendown.csv")
notee

,text
0,Admission Date: [**2119-5-4**] D...
1,Admission Date: [**2124-7-21**] ...
2,Admission Date: [**2162-3-3**] D...
3,Admission Date: [**2172-3-5**] D...
4,Admission Date: [**2112-12-8**] ...
5,Admission Date: [**2150-2-25**] ...
6,Admission Date: [**2118-8-10**] ...
7,Admission Date: [**2118-12-7**] ...
8,Admission Date: [**2125-2-9**] D...
9,Admission Date: [**2192-4-19**] ...


In [184]:
#Faker Replacement Stuff
"""
def replace_deidentified_chunks(text):
    # Define patterns to match
    patterns = {
        r'\[**Name \(NI\) **\]': fake.name,
        r'\[**Hospital\d+ **\]': lambda: fake.company() + " Hospital",
        r'\[**Last Name \(NamePattern\d+\) \d+**\]': fake.last_name,
        r'\[**First Name\d+ \(NamePattern\d+\) \d+**\]': fake.first_name,
        r'\[**Country \d+**\]': fake.country,
        r'\[**State \d+**\]': fake.state,
        r'\[**Location \(un\) \d+**\]': fake.city,
        r'\[**Company \d+**\]': fake.company,
        r'\[**\d{1,2}-\d{1,2}**\]': lambda: f"[**{fake.date(pattern='%m-%d')}**]",
        r'\[**Month \(only\) **\]': fake.month_name,
        r'\[**\d{4}**\]': lambda: f"[**{fake.year()}**]"
    }
    
    for pattern, replacement_func in patterns.items():
        text = re.sub(pattern, lambda _: replacement_func(), text)
    
    return text
"""

<>:2: SyntaxWarning: invalid escape sequence '\['
<>:2: SyntaxWarning: invalid escape sequence '\['
C:\Users\abobo8\AppData\Local\Temp\1\ipykernel_3728\3386246862.py:2: SyntaxWarning: invalid escape sequence '\['
  """


'\ndef replace_deidentified_chunks(text):\n    # Define patterns to match\n    patterns = {\n        r\'\\[**Name \\(NI\\) **\\]\': fake.name,\n        r\'\\[**Hospital\\d+ **\\]\': lambda: fake.company() + " Hospital",\n        r\'\\[**Last Name \\(NamePattern\\d+\\) \\d+**\\]\': fake.last_name,\n        r\'\\[**First Name\\d+ \\(NamePattern\\d+\\) \\d+**\\]\': fake.first_name,\n        r\'\\[**Country \\d+**\\]\': fake.country,\n        r\'\\[**State \\d+**\\]\': fake.state,\n        r\'\\[**Location \\(un\\) \\d+**\\]\': fake.city,\n        r\'\\[**Company \\d+**\\]\': fake.company,\n        r\'\\[**\\d{1,2}-\\d{1,2}**\\]\': lambda: f"[**{fake.date(pattern=\'%m-%d\')}**]",\n        r\'\\[**Month \\(only\\) **\\]\': fake.month_name,\n        r\'\\[**\\d{4}**\\]\': lambda: f"[**{fake.year()}**]"\n    }\n    \n    for pattern, replacement_func in patterns.items():\n        text = re.sub(pattern, lambda _: replacement_func(), text)\n    \n    return text\n'

In [185]:
oldset = open_file('sdoh_dataset.jsonl')

In [186]:
#replacemethod = oldset.replace("[**Name (NI) **]", fake.name())

print(oldset)


patterns_replacements = [
    (r'\[\*\*Hospital2 \*\*\]', '<HOSPITAL UNKNOWN>'),
    (r'\[\*\*Country \d+\*\*\]', '<COUNTRY UNKNOWN>'),
    (r'\[\*\*Name \(NI\) \*\*\]', '<NAME UNKNOWN>'),
    (r'\[\*\*known firstname \*\*\]', '<KNOWN FIRSTNAME>'),
    (r'\[\*\*last name \(namepattern1\) \d+\*\*\]', '<LASTNAME UNKNOWN>'),
    (r'\[\*\*first name8 \(namepattern2\) \d+\*\*\]', '<FIRSTNAME UNKNOWN>'),
    (r'\[\*\*last name \(namepattern1\) \*\*\]', '<LASTNAME UNKNOWN>'),
    (r'\[\*\*\d+\*\*\]', '<NUMBER UNKNOWN>'),
    (r'\[\*\*Initials \(NamePattern4\) \*\*\]', '<INITIALS UNKNOWN>'),
    (r'\[\*\*Last Name \(NamePattern4\) \*\*\]', '<LASTNAME UNKNOWN>'),
    (r'\[\*\*Company \d+\*\*\]', '<COMPANY UNKNOWN>'),
    (r'\[\*\*Month \(only\) \*\*\]', '<MONTH UNKNOWN>'),
    (r'\[\*\*Doctor Last Name \*\*\]-\[\*\*Doctor Last Name \*\*\]', '<DOCTOR LAST NAME UNKNOWN>-<DOCTOR LAST NAME UNKNOWN>'),
    (r'\[\*\*Location \(un\) \d+\*\*\]', '<LOCATION UNKNOWN>'),
    (r'\[\*\*Place of Living \*\*\]', '<PLACE OF LIVING UNKNOWN>')
]

text = oldset


for pattern, replacement in patterns_replacements:
    text = re.sub(pattern, replacement, text)

print(text)


{"messages": [{"role": "system", "content": "You are useful and helpful chatbot that helps generate synthetic clinical notes when asked."}, {"role": "user", "content": "is a nurse.\n\n\nPt is retired.\n\n\nPersonal history records alcohol use, tobacco use, and\n\ndrug abuse. He is a former medical student in a private\n\nstudio.\n\n\nFamily history records alcohol use, tobacco use, and\n\ndrug abuse. He is a former medical student in a privatestudio.\n\n\nDr. Kavanagh, [**Hospital1 **] at [**Hospital1 **].  Does not have a doctorate.\n\n[**Hospital1 **] does not have a doctorate, and has never\n\nbeen diagnosed with any form of alcohol abuse.\n\n\nFamily history records alcohol use, tobacco use, and\n\ndrug abuse. He is a former medical student in a privatestudio.\n\n\nFamily history records alcohol use, tobacco use, and\n\ndrug abuse. He is a former medical student in a privatestudio.\n\n\nFamily history records alcohol use,"}]}

{"messages": [{"role": "system", "content": "You are us